In [9]:
import scipy.io
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.spatial import distance
from geopy.distance import geodesic
import geopy.point as point
import tqdm
from tools import *

In [3]:
location = 'sahara_small'
meta_data = scipy.io.loadmat(f'./dataset/{location}_cell.mat')
cir_profile = meta_data[f'{location}_cell']['cir'][0][0]
dist = meta_data[f'{location}_cell']['dist'][0][0]

Y = meta_data[f'{location}_cell'][0][0]['tx'].T # coordination of agents (lat, lon)
RX = meta_data[f'{location}_cell'][0][0]['rx'].T
p_a_arr = Y
p_i_arr = RX

In [5]:
import scipy.stats as stats
from scipy.special import erf 
from math import isclose

In [6]:
def Q(x):
    '''Gaussian Q function'''
    return .5 - .5  * erf(x / np.sqrt(2))

global p_NL, p_E, eta, theta_max
p_NL = .5 # probability of all delays are from NLOS path
P_E = .5 # weights of NL,E in NL,E and NL,M sources
eta = 1e6 # parameter for exponential distribution of NL,M, depending on channel
theta_max = max(delay_set)

def p_ai(a_i, L):
    if a_i == 0:
        return p_NL
    elif a_i > 0 and a_i <= L:
        return (1 - p_NL) / L
    else:
        return 0

def f_L(theta, p_a, p_i):
    return 1 if isclose(theta, geodesic(p_a, p_i).m / 3e8, rel_tol=.05) else 0

def f_NL(theta, p_a, p_i):
    theta_min = geodesic(p_a, p_i).m / 3e8
    theta_max = max(delay_set) # predefined maximum possible delay
    print(f_NLE(theta, theta_max), f_NLM(theta, theta_min, theta_max))
    return P_E * f_NLE(theta, theta_max) + (1 - P_E) * f_NLM(theta, theta_min, theta_max)

def f_NLM(theta, theta_min, theta_max):
    return eta * np.exp(-eta * (theta - theta_min)) / (1 - np.exp(-eta * (theta_max - theta_min))) \
        if theta >= theta_min and theta < theta_max \
            else 0

def f_NLE(theta, theta_max):
    return 1 / theta_max if theta >= 0 and theta < theta_max else 0

def f_anchor_i_ai(theta_arr, p_a, p_i, a_i):
    '''conditional prob of anchor i given p_a, p_i, a_i 
    theta_arr reprents the entire theta arr in CIR for anchor i'''
    return np.prod([f_NL(theta, p_a, p_i) for theta in theta_arr]) if a_i == 0 \
        else f_L(theta_arr[a_i]) * np.prod([f_NL(theta, p_a, p_i) for theta in theta_arr if theta != theta_arr[a_i]])

def f_anchor_i(theta_arr, p_a, p_i):
    # conditional prob of anchor i given p_a, p_i
    L = len(theta_arr)
    # LOS term
    los_term = 0
    for a_i in range(L):
        los_term += f_L(theta_arr[a_i], p_a, p_i) * \
            np.prod([f_NL(theta, p_a, p_i) for theta in theta_arr if theta != theta_arr[a_i]])
        # if f_L(theta_arr[a_i], p_a, p_i) == 1:
        #     print('los term is', los_term, np.prod([f_NL(theta, p_a, p_i) for theta in theta_arr if theta != theta_arr[a_i]]))
    return p_NL * np.prod([f_NL(theta, p_a, p_i) for theta in theta_arr]) + \
        (1 - p_NL) / L * los_term

def f_anchor(theta_mtx, p_a, p_i_arr):
    # theta_mtx shape: N * L - num of stations by num of ray traces per station
    return np.prod([f_anchor_i(theta_mtx[i], p_a, p_i_arr[i]) for i in range(len(theta_mtx))])

def m_L(p_a, p_i, tau, sigma):
    return stats.norm(tau, sigma).pdf(geodesic(p_a, p_i).m / 3e8)

def m_NL(theta_min, theta_max, tau, sigma):
    return P_E / theta_max * ( Q(-tau/sigma)- Q((theta_max - tau)/sigma) ) + \
            (1 - P_E) * eta * np.exp(eta * theta_min - eta * tau + .5 * eta**2 * sigma**2) * \
                (Q((theta_min - tau + eta*sigma**2) / sigma) - Q((theta_max - tau + eta*sigma**2) / sigma))

def m_i(p_a, p_i, theta_max, tau_arr, sigma_arr):
    '''
    message update for anchor i
    '''
    L = len(tau_arr)
    theta_min = geodesic(p_a, p_i).m / 3e8
    # LOS term
    los_term = 0
    for a_i in range(L):
        los_term += m_L(p_a, p_i, tau_arr[a_i], sigma_arr[a_i]) * \
            np.product([m_NL(theta_min, theta_max, tau_arr[j], sigma_arr[j]) for j in range(0, L) if j != a_i])
    
    return p_NL * np.prod([m_NL(theta_min, theta_max, tau, sigma) for tau, sigma in zip(tau_arr, sigma_arr)]) + \
        (1 - p_NL) / L * los_term


def f_pa(f_p, p_a, p_i_arr, theta_max, tau_mtx, sigma_mtx):
    return f_p * np.product([m_i(p_a, p_i, theta_max, tau_arr, sigma_arr) \
        for p_i, tau_arr, sigma_arr in zip(p_i_arr, tau_mtx, sigma_mtx)])
    ...

def g(p_a, p_i_arr, tau_mtx, sigma_mtx):
    '''
    Gaussian mixture rings of one agent
    '''
    N = len(p_i_arr)

    def g_i(p_a, p_i, tau_arr, sigma_arr):
        '''
        Gaussian mixture rings
        '''
        L = len(tau_arr)

        return 1 / L * np.sum([m_L(p_a, p_i, tau,  sigma) for tau, sigma in zip(tau_arr, sigma_arr)])

    return np.sum([g_i(p_a, p_i, tau_arr, sigma_arr) for p_i, tau_arr, sigma_arr in zip(p_i_arr, tau_mtx, sigma_mtx)])

NameError: name 'delay_set' is not defined

# Sampling agents

In [7]:
class AgentSampling(object):
    def __init__(self,circle=360, S=1000):
        self.circle = circle
        self.S = S

    def sampling(self, p_a, p_i_arr, tau, sigma):
        
        self.p_a = p_a
        self.tau = tau
        self.sigma = sigma
        self.p_i_arr = p_i_arr

        p_a_s = []
        weight = []
        N = len(self.p_i_arr)

        for s in range(self.S):
            i = np.random.choice(N) # sample over stations
            j = np.random.choice(len(self.tau[i])) # sample over traces between anchor and agent

            deg = np.random.choice(self.circle)
            p_i = self.p_i_arr[i]

            origin = point.Point(p_i)
            
            tmp = np.random.normal(self.tau[i][j], self.sigma[i][j])
            d = tmp * 3e8

            f_p = stats.norm(self.tau[i][j], self.sigma[i][j].T).pdf(tmp)
            
            p_s_loc = geodesic(d/1000).destination(origin, deg)
            p_s = np.array([p_s_loc.latitude, p_s_loc.longitude])
            fpa = f_pa(f_p, p_s, self.p_i_arr, theta_max, self.tau, self.sigma)
            ga = g(p_s, self.p_i_arr, self.tau, self.sigma)

            weight.append(np.abs(fpa / ga))
            p_a_s.append(p_s)

        self.p_a_s = np.array(p_a_s)
        self.weight = np.array(weight) / np.sum(weight)
        self.p_estim = np.multiply(self.weight, self.p_a_s.T).sum(axis=1)
        self.error = geodesic(self.p_a, self.p_estim)
        # self.p_estim = self.p_a_s.mean(axis=0)


    def visualize(self, ax):
        ax.scatter(self.p_a_s[:, 0], self.p_a_s[:, 1], c=self.weight, marker='o')
        ax.scatter(self.p_i_arr[:, 0], self.p_i_arr[:, 1])
        ax.scatter(self.p_a[0], self.p_a[1], s=320, marker='*')
        ax.set_title(f'distance error {self.error} m')
        ax.scatter(self.p_estim[0], self.p_estim[1], s=350, marker='x', color='r')
        # plot_agent(self.p_a_s)
        # plot_agent(self.p_i_arr)
        # plot_agent(self.p_a)

In [ ]:
n_a = 25
idx = np.random.choice(len(p_a_arr), n_a)
p_a_set = p_a_arr[idx]
fig, axs = plt.subplots(int(np.sqrt(n_a)), int(np.sqrt(n_a)), figsize=(20, 20))
axs = axs.flatten()
for k in tqdm(range(len(p_a_set))):
    p_a = p_a_set[k]
    agentsampling = AgentSampling(S=1000)
    agentsampling.sampling(p_a, p_i_arr, theta_mtx[idx[k]], sigma_mtx[idx[k]])
    agentsampling.visualize(axs[k])